# Trabalho de Algoritmos Genéticos

## Problema Do Empacotamento (BPP do inglês *"Bin Packing Problem"*)

Dada uma quantidade inteira e positiva de pacotes/depósitos de capacidade $C$ e um set de $M$ itens  $I = [I_1, \cdots, I_M]$ de tamanhos $S = [S_1,\cdots,S_M]$, o problema consiste em empacotar todos os itens nos pacotes, de modo a não exceder a capacidade $C$, **minimizando** a quantidade $N$ de pacotes utilizados.

## Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

import pygad
import tensorflow.keras

import numpy as np
import pandas as pd
from itertools import combinations

## Modelando o Problema

- Gene: É um pacote.
    - É representado por um set de inteiros referentes à cada cada item;
- Indivíduo: É um conjunto de pacotes, que somam um total de $M$ elementos, cujos respectivos pesos não infringem a capacidade $C$.

In [2]:
def f(individuo):
    return len(individuo)

In [3]:
def FF(individuo, item, C, S):

    if individuo == []:
        individuo.append([item])
        return

    added = False
    for pacote in individuo:
        if sum([x for i, x in enumerate(S) if i in pacote]) + S[item] <= C:
            pacote.append(item)
            pacote.sort()
            added = True
            break

    if not added:
        individuo.append([item])
            

def FFD(individuo, itens, C, S):
    ordenados = [x[0] for x in sorted(
        [(itens[i], S[i]) for i in range(0, len(itens))], 
        key = lambda x: x[1],
        reverse=True)]
    
    for item in ordenados:
        FF(individuo, item, C, S)


def MBS(individuo, item, C, S):

    if individuo == []:
        individuo.append([item])
        return

    added = False
    melhor_diferenca = C
    indice_melhor = 0
    for I, pacote in enumerate(individuo):
        diferenca = C - (sum([x for i, x in enumerate(S) if i in pacote]) + S[item])
        if diferenca < melhor_diferenca and diferenca >= 0:
            melhor_diferenca = diferenca
            indice_melhor = I
            added = True

    if added:
        individuo[indice_melhor] += (item)
        individuo[indice_melhor].sort()
    else:
        individuo.append([item])

def FleszarMBS(individuo, itens, C, S):
    ordenados = [x[0] for x in sorted(
        [(itens[i], S[i]) for i in range(0, len(itens))], 
        key = lambda x: x[1],
        reverse=True)]
    for it in ordenados:
        MBS(individuo, it, C, S)

In [4]:
def add(pacote, item, C, S):
    if (sum([x for i, x in enumerate(S) if i in pacote]) + S[item]) <= C:
        pacote.append(item)

In [35]:
def P2F(individuo, Tx, C, S, Tb=True):
    if Tb:
        for pacote in Tx:
            for pct in individuo:
                if sum([x for i, x in enumerate(S) if i in pct]) + sum([x for i, x in enumerate(S) if i in pacote]) <= C:
                    pct += pacote
                    pct.sort()
                    Tx.remove(pacote)
                    break
        # Precisa mesmo disso? <- creio que não
        ordenados = [x[0] for x in sorted(
            [(i, sum(
                [k for j,k in enumerate(S) if  j in i])
             ) for i in Tx], key = lambda x: x[1], reverse = True)]
        for ordn in ordenados:
            individuo.append(ordn)
            Tx.remove(ordn)
    else:
        for item in Tx:
            FF(individuo, item, C, S)
            Tx.remove(item)

def substituicao(individuo, Ta, Tb, C, S):
    Tc = [list(x) for x in combinations(Ta, 2)]

    S_vec = []
    for pacote in individuo:
        Sv = []
        Sv += [list(x) for x in combinations(pacote, 1)]
        Sv += [list(x) for x in combinations(pacote, 2)]
        Sv += [list(x) for x in combinations(pacote, 3)]

        S_vec += (Sv)

    f_ind = f(individuo)
    lista_candidatos = []
    for candidato in S_vec:
        nTa = Ta.copy()
        nTa = [x for x in nTa if x not in candidato]
        for pct in individuo:
            nInd = individuo.copy()
            nInd.remove(pct)

            nInd.append(candidato)
            for item in pct:
                nTa.append(item)
            
            tam_total = 0
            for pct in nInd:
                tam_total += sum([x for i, x in enumerate(S) if i in pct])
            lista_candidatos.append(tuple((tam_total, nTa, Tb.copy(), nInd)))


    for candidato in Tb:
        nTb = Tb.copy()
        nTb.remove(candidato)
        nTa = Ta.copy()
        for pct in individuo:
            nInd = individuo.copy()
            nInd.remove(pct)

            nInd.append(candidato)
            for item in pct:
                nTa.append(item)

            tam_total = 0
            for pct in nInd:
                tam_total += sum([x for i, x in enumerate(S) if i in pct])
            lista_candidatos.append(tuple((tam_total, nTa, nTb, nInd)))

    melhor_candidato = max(lista_candidatos, key = lambda x: x[0])
    
    P2F(melhor_candidato[-1], Tb, C, S)
    P2F(melhor_candidato[-1], Ta, C, S, Tb=False)
                
        
        
def cruzamento(pai1, pai2, C, S):
    return fcruzamento(pai1, pai2, C, S), fcruzamento(pai2, pai1, C, S)

def fcruzamento(pai1, pai2, C, S):
    idx1 = [1,3] # rand 2 de cima
    idx2 = [3]   # rand 1 de baixo mas sem elementos de pai1[indices1]

    offset = []
    offset += [pacote for indice, pacote in enumerate(pai1) if indice in idx1]
    offset += [pacote for indice, pacote in enumerate(pai2) if indice in idx2]
    
    Ta = []
    Tb = []

    # pacotes remanescentes em pai2
    for pacote in [pct for indice, pct in enumerate(pai2) if indice not in idx2]:
        novo_pacote = []
        for item in pacote:
            if not any(item in o_pct for o_pct in offset):
                novo_pacote.append(item)

        if len(novo_pacote) > 1:
            Tb.append(novo_pacote)
        elif len(novo_pacote) == 1:
            Ta.append(novo_pacote[0])
    

    substituicao(offset, Ta, Tb, C, S)

    return offset

In [6]:
p1 = [[0,1,3,10],[2,9,11],[5,7,13,15],[4,6,14],[8,12]]
p2 = [[3,4,12],[6,7,11],[9,10],[1,5,8],[2,13,14]]

In [36]:
for x in cruzamento(p1, p2, 40, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]):
    print(x)

[[2, 3, 9, 11, 12], [4, 6, 7, 14], [1, 5, 8]]
[[0, 3, 6, 7, 10, 11], [1, 5, 8], [4, 6, 7, 14]]
